# MAPSI Tutorial

## Overview
MAPSI (MzML And Processed Spectra Interface) is designed to aid the user in combining and viewing mzML files and their corresponding PSM, peptide, and protein quantification files into a multiIndexed pandas dataframe.

Depending on whether you are using MetaMorpheus or MSFragger files, you will want to import the corresponding parser script. In this tutorial I will import both parser scripts.

In [1]:
from MSFragger_Parser import parse_files as msfragger_parser
from MetaMorpheus_Parser import parse_files as MetaMorpheus_parser

In [2]:
import pandas as pd

## Test Files

In [3]:
# mzML test file
mzml_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\Ex_Auto_J3_30umTB_2ngQC_60m_1.mzML"

# MSFragger test files
msfragger_psm_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\msfragger\\psm1.tsv"
msfragger_peptide_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\msfragger\\combined_peptide.tsv"
msfragger_protein_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\msfragger\\combined_protein.tsv"

# MetaMorpheus test files
mzml_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\Ex_Auto_J3_30umTB_2ngQC_60m_1.mzML"
mm_psm_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\Ex_Auto_J3_30umTB_2ngQC_60m_1-calib_PSMs.psmtsv"
mm_peptideQ_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\AllQuantifiedPeptides.tsv"
mm_protein_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\AllQuantifiedProteinGroups.tsv"

# output test file
outfile = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\msfragger\\tester.tsv"

## Inputs
#### input_files
You may input a list of 1 to 4 files in any order. If you only want to read in one file, it can be inputted as a string.

In [4]:
# let's try combining a psm and peptide quantification file
psm_and_peptide_df = msfragger_parser(input_files=[msfragger_peptide_file_path, msfragger_psm_file_path])
psm_and_peptide_df.head()

,,,File Name,Modified Peptide,Charge,Retention,Observed Mass,Calibrated Observed Mass,Observed M/Z,Calibrated Observed M/Z,Calculated Peptide Mass,Calculated M/Z,...,Ex_Auto_K13_30umTA_02ngQC_60m_1 Intensity,Ex_Auto_K13_30umTA_02ngQC_60m_2 Intensity,Ex_Auto_W17_30umTA_02ngQC_60m_3 Intensity,Ex_Auto_W17_30umTA_02ngQC_60m_4 Intensity,Ex_Auto_J3_30umTB_02ngQC_60m_1 MaxLFQ Intensity,Ex_Auto_J3_30umTB_02ngQC_60m_2 MaxLFQ Intensity,Ex_Auto_K13_30umTA_02ngQC_60m_1 MaxLFQ Intensity,Ex_Auto_K13_30umTA_02ngQC_60m_2 MaxLFQ Intensity,Ex_Auto_W17_30umTA_02ngQC_60m_3 MaxLFQ Intensity,Ex_Auto_W17_30umTA_02ngQC_60m_4 MaxLFQ Intensity
Protein Accession,Peptide,Scan Number,,,,,,,,,,,,,,,,,,,,,
A0A096LP55,EQCEQLEK,5237,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,EQCEQLEK,2,1776.8865,1062.4706,1062.4664,532.2426,532.2405,1062.4651,532.2398,...,0.00,0.00,0.00,0.0,219563.420,234511.55,0.00,0.00,0.00,0.0
A6NHQ2,TNIIPVLEDAR,14755,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,NaN,2,4218.5518,1239.6882,1239.6842,620.8514,620.8494,1239.6823,620.8484,...,523321.72,649737.94,814822.10,1081377.4,560720.750,659080.60,523321.72,649737.94,814822.10,1081377.4
A6NKT7,NSIPEPIDPLFK,16986,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,NaN,2,4795.0662,1368.7386,1368.7316,685.3766,685.3731,1368.7288,685.3717,...,0.00,64618.39,0.00,0.0,33486.430,34693.87,0.00,64618.39,0.00,0.0
A8MWD9,GNSIIMLEALER,18334,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,NaN,2,5152.8337,1344.7134,1344.7035,673.3640,673.3590,1344.7070,673.3608,...,409638.97,376445.30,231965.92,487477.6,501898.060,230573.27,414935.78,203264.50,234965.34,493780.9
B0I1T2,DFLFQDFK,18706,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,NaN,2,5252.6033,1058.5103,1058.5061,530.2624,530.2603,1058.5072,530.2609,...,0.00,0.00,0.00,0.0,59026.492,104090.06,0.00,0.00,0.00,0.0


In [5]:
# we can also just load data from one file
psm_df = msfragger_parser(input_files=msfragger_psm_file_path)
psm_df.head()

converting input_files from a string into list


,,,File Name,Modified Peptide,Prev AA,Next AA,Peptide Length,Charge,Retention,Observed Mass,Calibrated Observed Mass,Observed M/Z,...,Protein Start,Protein End,Intensity,Assigned Modifications,Observed Modifications,Is Unique,Protein,Entry Name,Gene,Protein Description
Protein Accession,Peptide,Scan Number,,,,,,,,,,,,,,,,,,,,,
A0A096LP55,EQCEQLEK,5237,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,EQCEQLEK,R,C,8,2,1776.8865,1062.4706,1062.4664,532.2426,...,35,42,200838.400,3C(57.0215),NaN,False,sp|A0A096LP55|QCR6L_HUMAN,QCR6L_HUMAN,UQCRHL,"Cytochrome b-c1 complex subunit 6-like, mitoch..."
A6NHQ2,TNIIPVLEDAR,14755,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,NaN,R,H,11,2,4218.5518,1239.6882,1239.6842,620.8514,...,220,230,517661.100,NaN,NaN,True,sp|A6NHQ2|FBLL1_HUMAN,FBLL1_HUMAN,FBLL1,rRNA/tRNA 2'-O-methyltransferase fibrillarin-l...
A6NKT7,NSIPEPIDPLFK,16986,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,NaN,K,H,12,2,4795.0662,1368.7386,1368.7316,685.3766,...,621,632,30740.525,NaN,NaN,False,sp|A6NKT7|RGPD3_HUMAN,RGPD3_HUMAN,RGPD3,RanBP2-like and GRIP domain-containing protein 3
A8MWD9,GNSIIMLEALER,18334,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,NaN,R,-,12,2,5152.8337,1344.7134,1344.7035,673.3640,...,64,75,445926.780,NaN,NaN,False,sp|A8MWD9|RUXGL_HUMAN,RUXGL_HUMAN,SNRPGP15,Putative small nuclear ribonucleoprotein G-lik...
B0I1T2,DFLFQDFK,18706,interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml,NaN,R,R,8,2,5252.6033,1058.5103,1058.5061,530.2624,...,537,544,54186.293,NaN,NaN,True,sp|B0I1T2|MYO1G_HUMAN,MYO1G_HUMAN,MYO1G,Unconventional myosin-Ig


  Note that MAPSI cannot process more than one of each file type. If you would like to combine several files of the same type (ex: multiple mzML and PSM files), it is recommended that you run corresponding files through MAPSI and then concatenate the resulting dataframes.   
  ```python
  list_of_mzml_file_paths = ['mzml_1', 'mzml_2', 'mzml_3']
  list_of_psm_file_paths = ['psm_1', 'psm_2', 'psm_3']
  # iterate over the mzML and PSM files 
  list_of_joined_dataframes = []
  for index, mzml_file_path in enumerate(list_of_mzml_file_paths):
    psm_file_path = list_of_psm_file_paths[index]
    # join data from the mzML and PSM files and append it to the 'list_of_joined_dataframes'
    new_df = msfragger_parser(input_files=[mzml_file_path, psm_file_path])
    list_of_joined_dataframes.append(new_df)
  # loop through the new_df's and concatenate them
  joined_df = pd.concat(list_of_joined_dataframes)
  ```

MAPSI recognizes file types by the file extension and name. If MAPSI is having trouble recognizing your files, make sure that:
1. The mzML file extension is `.mzml`
2. The PSM file name contains `psm`
3. The peptide file name contains `peptide` or `pep`
4. The protein file name contains `protein` or `prot`

#### output_file_path
This is an optional parameter which allows you to save your dataframe as a `.tsv` file. 

In [6]:
# ex:
msfragger_parser(input_files=[msfragger_protein_file_path, msfragger_peptide_file_path], output_file_path=outfile).head()

Dataframe saved.


,,Prev AA,Next AA,Start,End,Peptide Length,Charges,Protein Description,Mapped Genes,Mapped Proteins,Ex_Auto_J3_30umTB_02ngQC_60m_1 Spectral Count_peptide,...,Ex_Auto_K13_30umTA_02ngQC_60m_2 MaxLFQ Unique Intensity,Ex_Auto_W17_30umTA_02ngQC_60m_3 MaxLFQ Unique Intensity,Ex_Auto_W17_30umTA_02ngQC_60m_4 MaxLFQ Unique Intensity,Ex_Auto_J3_30umTB_02ngQC_60m_1 MaxLFQ Total Intensity,Ex_Auto_J3_30umTB_02ngQC_60m_2 MaxLFQ Total Intensity,Ex_Auto_K13_30umTA_02ngQC_60m_1 MaxLFQ Total Intensity,Ex_Auto_K13_30umTA_02ngQC_60m_2 MaxLFQ Total Intensity,Ex_Auto_W17_30umTA_02ngQC_60m_3 MaxLFQ Total Intensity,Ex_Auto_W17_30umTA_02ngQC_60m_4 MaxLFQ Total Intensity,Indistinguishable Proteins
Protein Accession,Peptide,,,,,,,,,,,,,,,,,,,,,
A0A096LP55,EQCEQLEK,R,C,35,42,8,2,"Cytochrome b-c1 complex subunit 6-like, mitoch...",UQCRH,sp|P07919|QCR6_HUMAN,1,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,sp|P07919|QCR6_HUMAN
A6NDG6,TILTLTGVSTLGDVK,K,N,276,290,15,2,Glycerol-3-phosphate phosphatase,NaN,NaN,0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,NaN
A6NHL2,SFGGGTGSGFTSLLMER,R,L,147,163,17,2,Tubulin alpha chain-like 3,NaN,NaN,0,...,0.0,0.0,0.0,3170630.80,2816038.0,3657354.00,3326739.80,2417819.5,4331914.5,NaN
A6NHQ2,TNIIPVLEDAR,R,H,220,230,11,2,rRNA/tRNA 2'-O-methyltransferase fibrillarin-l...,NaN,NaN,1,...,0.0,0.0,0.0,629399.44,553218.6,720522.44,609172.75,678163.0,1068284.8,NaN
A6NKT7,NSIPEPIDPLFK,K,H,621,632,12,2,RanBP2-like and GRIP domain-containing protein 3,"RANBP2, RGPD1, RGPD2, RGPD4, RGPD5, RGPD8","sp|O14715|RGPD8_HUMAN, sp|P0DJD0|RGPD1_HUMAN, ...",1,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,"sp|O14715|RGPD8_HUMAN, sp|P0DJD0|RGPD1_HUMAN, ..."


#### columns_to_keep
This is an optional parameter which allows you to input a list of columns you would like to see the data for (MAPSI will also take a str if you only wish to input one column). It is recommended that you check the column names from your original columns before running MAPSI.

In [7]:
my_dataframe = MetaMorpheus_parser(input_files=[mm_peptideQ_file_path, mm_psm_file_path, mm_protein_file_path], columns_to_keep=['Scan Number', 'Protein Accession', 'Peptide', 'Gene Name', 'Base Sequence', 'Score'])
my_dataframe.head()

Your inputs: [['C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\AllQuantifiedPeptides.tsv', 'C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\Ex_Auto_J3_30umTB_2ngQC_60m_1-calib_PSMs.psmtsv', 'C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\AllQuantifiedProteinGroups.tsv'], None, ['Scan Number', 'Protein Accession', 'Peptide', 'Gene Name', 'Base Sequence', 'Score'], None, None, None, None]
.tsv
c:\users\sarah curtis\onedrive - byu\documents\single cell team documents\api_dev\metam\2ng\allquantifiedpeptides
.psmtsv
c:\users\sarah curtis\onedrive - byu\documents\single cell team documents\api_dev\metam\2ng\ex_auto_j3_30umtb_2ngqc_60m_1-calib_psms
.tsv
c:\users\sarah curtis\onedrive - byu\documents\single cell team documents\api_dev\metam\2ng\allquantifiedproteingroups


c:\Users\Sarah Curtis\Documents\Single Cell\MAPSIN\Parser_code\MetaMorpheus_Parser.py:64: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  psm_dataframe = pd.read_table(psm_file_path, delimiter='\t')


Gene Name  \
Protein Accession Peptide         Scan Number                                 
A0A075B6X5        EPELLLK         24419                      primary:TRAV18   
                                  24592                      primary:TRAV18   
A0A096LP49        LQDVAGR         6514                      primary:CCDC187   
A0A0B4J2D5|P0DPI2 GVEVTVGHEQEEGGK 8901         primary:GATD3B|primary:GATD3   
                  NLSTFAVDGK      16610        primary:GATD3B|primary:GATD3   

                                                 Base Sequence   Score  
Protein Accession Peptide         Scan Number                           
A0A075B6X5        EPELLLK         24419                EPELLLK   6.399  
                                  24592                EPELLLK   6.293  
A0A096LP49        LQDVAGR         6514                 LQDVAGR   6.208  
A0A0B4J2D5|P0DPI2 GVEVTVGHEQEEGGK 8901         GVEVTVGHEQEEGGK  11.070  
                  NLSTFAVDGK      16610             NLSTFAVDGK   9.268


If one of the inputted columns is not found in the database, the entire database will be returned to ensure that you are not missing any important information. To manually select columns to keep, follow the outline below:    
```python
example_columns_to_keep = ['Column_1', 'Column_2', 'Column_3']
my_dataframe = my_dataframe[example_columns_to_keep]
```

In [8]:
modified_df = my_dataframe[['Gene Name', 'Score']]
modified_df.head()

Gene Name  \
Protein Accession Peptide         Scan Number                                 
A0A075B6X5        EPELLLK         24419                      primary:TRAV18   
                                  24592                      primary:TRAV18   
A0A096LP49        LQDVAGR         6514                      primary:CCDC187   
A0A0B4J2D5|P0DPI2 GVEVTVGHEQEEGGK 8901         primary:GATD3B|primary:GATD3   
                  NLSTFAVDGK      16610        primary:GATD3B|primary:GATD3   

                                                Score  
Protein Accession Peptide         Scan Number          
A0A075B6X5        EPELLLK         24419         6.399  
                                  24592         6.293  
A0A096LP49        LQDVAGR         6514          6.208  
A0A0B4J2D5|P0DPI2 GVEVTVGHEQEEGGK 8901         11.070  
                  NLSTFAVDGK      16610         9.268

#### rows_to_keep: (scans_to_keep, peptides_to_keep, proteins_to_keep)
In addition to selecting columns to keep, you can also select specific rows to keep in your dataframe. This is split into 3 optional parameters which allow you to select rows based on scan number, peptide sequence, and/or protein accession. Note that these parameters must be entered as a `list` of str or a `str`.

In [9]:
scans = ['16819', '17761', '8942', '9686', '10645', '17633', '17638']
proteins = ['B5ME19', 'E9PAV3']
msfragger_parser(input_files=[msfragger_psm_file_path, msfragger_peptide_file_path], scans_to_keep=scans, proteins_to_keep=proteins)

File Name  \
Protein Accession Peptide           Scan Number                                                    
B5ME19            CLEEFELLGK        16819        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  ELLGQGLLLR        17761        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  IMQNTDPHSQEYVEHLK 8942         interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  LNEILQAR          9686         interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
E9PAV3            IEDLSQQAQLAAAEK   10645        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  NILFVITKPDVYK     17633        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                                    17638        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   

                                                Modified Peptide  Charge  \
Protein Accession Peptide           Scan Number                            
B5ME19            CLEEFELLGK        16819             CLEEFELLGK       2   
                  ELLGQGLLLR        17761                    NaN       2   
                  IMQNTDPHSQEYVEHLK 8942                     NaN       4   
                  LNEILQAR          9686                     NaN       2   
E9PAV3            IEDLSQQAQLAAAEK   10645                    NaN       2   
                  NILFVITKPDVYK     17633                    NaN       3   
                                    17638                    NaN       2   

                                                 Retention  Observed Mass  \
Protein Accession Peptide           Scan Number                             
B5ME19            CLEEFELLGK        16819        4750.5885      1236.6061   
                  ELLGQGLLLR        17761        5000.1031      1110.6814   
                  IMQNTDPHSQEYVEHLK 8942         2727.5402      2067.9734   
                  LNEILQAR          9686         2918.0127       955.5461   
E9PAV3            IEDLSQQAQLAAAEK   10645        3163.9378      1613.8329   
                  NILFVITKPDVYK     17633        4965.9947      1549.8988   
                                    17638        4967.3657      1548.8954   

                                                 Calibrated Observed Mass  \
Protein Accession Peptide           Scan Number                             
B5ME19            CLEEFELLGK        16819                       1236.6012   
                  ELLGQGLLLR        17761                       1110.6769   
                  IMQNTDPHSQEYVEHLK 8942                        2067.9666   
                  LNEILQAR          9686                         955.5394   
E9PAV3            IEDLSQQAQLAAAEK   10645                       1613.8262   
                  NILFVITKPDVYK     17633                       1549.8953   
                                    17638                       1548.8916   

                                                 Observed M/Z  \
Protein Accession Peptide           Scan Number                 
B5ME19            CLEEFELLGK        16819            619.3103   
                  ELLGQGLLLR        17761            556.3480   
                  IMQNTDPHSQEYVEHLK 8942             518.0006   
                  LNEILQAR          9686             478.7803   
E9PAV3            IEDLSQQAQLAAAEK   10645            807.9237   
                  NILFVITKPDVYK     17633            517.6402   
                                    17638            775.4550   

                                                 Calibrated Observed M/Z  \
Protein Accession Peptide           Scan Number                            
B5ME19            CLEEFELLGK        16819                       619.3079   
                  ELLGQGLLLR        17761                       556.3457   
                  IMQNTDPHSQEYVEHLK 8942                        517.9989   
                  LNEILQAR          9686                        478.7770   
E9PAV3            IEDLSQQAQLAAAEK   10645                       807.920

If one of the inputs is not found in the specified row, the dataframe will not be filtered based off of that row.

In [10]:
scans_1 = ['16819', '17761', '8942', '9686', '10645', '17633', '17638', '10000']
proteins_1 = ['B5ME19', 'E9PAV3']
selected_rows_df = msfragger_parser(input_files=[msfragger_psm_file_path, msfragger_peptide_file_path], scans_to_keep=scans_1, proteins_to_keep=proteins_1)
selected_rows_df

The following scans were not found in the dataframe: [10000]
To ensure that you have all the information needed, the dataframe will not be filted by Scan Number


File Name  \
Protein Accession Peptide           Scan Number                                                    
B5ME19            CLEEFELLGK        16819        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  ELLGQGLLLR        17761        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  GCILTLVER         15367        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  GTEITHAVVIK       9078         interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  IMQNTDPHSQEYVEHLK 8942         interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  LNEILQAR          9686         interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  TCHSFIINEK        9463         interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
E9PAV3            IEDLSQQAQLAAAEK   10645        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  NILFVITKPDVYK     17633        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                                    17638        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                                    17641        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  SPASDTYIVFGEAK    14235        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   

                                                Modified Peptide  Charge  \
Protein Accession Peptide           Scan Number                            
B5ME19            CLEEFELLGK        16819             CLEEFELLGK       2   
                  ELLGQGLLLR        17761                    NaN       2   
                  GCILTLVER         15367              GCILTLVER       2   
                  GTEITHAVVIK       9078                     NaN       2   
                  IMQNTDPHSQEYVEHLK 8942                     NaN       4   
                  LNEILQAR          9686                     NaN       2   
                  TCHSFIINEK        9463              TCHSFIINEK       3   
E9PAV3            IEDLSQQAQLAAAEK   10645                    NaN       2   
                  NILFVITKPDVYK     17633                    NaN       3   
                                    17638                    NaN       2   
                                    17641                    NaN       3   
                  SPASDTYIVFGEAK    14235                    NaN       2   

                                                 Retention  Observed Mass  \
Protein Accession Peptide           Scan Number                             
B5ME19            CLEEFELLGK        16819        4750.5885      1236.6061   
                  ELLGQGLLLR        17761        5000.1031      1110.6814   
                  GCILTLVER         15367        4375.2910      1059.5825   
                  GTEITHAVVIK       9078         2762.2254      1166.6719   
                  IMQNTDPHSQEYVEHLK 8942         2727.5402      2067.9734   
                  LNEILQAR          9686         2918.0127       955.5461   
                  TCHSFIINEK        9463         2860.9772      1247.6010   
E9PAV3            IEDLSQQAQLAAAEK   10645        3163.9378      1613.8329   
                  NILFVITKPDVYK     17633        4965.9947      1549.8988   
                                    17638        4967.3657      1548.8954   
                                    17641        4968.1137      1548.8992   
                  SPASDTYIVFGEAK    14235        4085.1155      1483.7235   

                                                 Calibrated Observed Mass  \
Protein Accession Peptide           Scan Number                             
B5ME19            CLEEFELLGK        16819                       1236.6012   
                  ELLGQGLLLR        17761                       1110.6769   
                  GCILTLVER         15367                       1059.5787   
                  GTEITHAVVIK       9078                        1166.6656   
                  IMQNTDPHSQEYVEHLK 8942                        2067.9666   
   

If you wish to manually correct this, you can do it as follows:

In [11]:
scans_corrected = [16819, 17761, 8942, 9686, 10645, 17633, 17638]
# Note that we need to reset the index in order to store the data into columns/rows
corrected_selected_rows_df = selected_rows_df.reset_index()
corrected_selected_rows_df = corrected_selected_rows_df.loc[corrected_selected_rows_df['Scan Number'].isin(scans_corrected)]
corrected_selected_rows_df = corrected_selected_rows_df.set_index(['Protein Accession', 'Peptide', 'Scan Number'])
corrected_selected_rows_df

File Name  \
Protein Accession Peptide           Scan Number                                                    
B5ME19            CLEEFELLGK        16819        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  ELLGQGLLLR        17761        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  IMQNTDPHSQEYVEHLK 8942         interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  LNEILQAR          9686         interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
E9PAV3            IEDLSQQAQLAAAEK   10645        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                  NILFVITKPDVYK     17633        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   
                                    17638        interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml   

                                                Modified Peptide  Charge  \
Protein Accession Peptide           Scan Number                            
B5ME19            CLEEFELLGK        16819             CLEEFELLGK       2   
                  ELLGQGLLLR        17761                    NaN       2   
                  IMQNTDPHSQEYVEHLK 8942                     NaN       4   
                  LNEILQAR          9686                     NaN       2   
E9PAV3            IEDLSQQAQLAAAEK   10645                    NaN       2   
                  NILFVITKPDVYK     17633                    NaN       3   
                                    17638                    NaN       2   

                                                 Retention  Observed Mass  \
Protein Accession Peptide           Scan Number                             
B5ME19            CLEEFELLGK        16819        4750.5885      1236.6061   
                  ELLGQGLLLR        17761        5000.1031      1110.6814   
                  IMQNTDPHSQEYVEHLK 8942         2727.5402      2067.9734   
                  LNEILQAR          9686         2918.0127       955.5461   
E9PAV3            IEDLSQQAQLAAAEK   10645        3163.9378      1613.8329   
                  NILFVITKPDVYK     17633        4965.9947      1549.8988   
                                    17638        4967.3657      1548.8954   

                                                 Calibrated Observed Mass  \
Protein Accession Peptide           Scan Number                             
B5ME19            CLEEFELLGK        16819                       1236.6012   
                  ELLGQGLLLR        17761                       1110.6769   
                  IMQNTDPHSQEYVEHLK 8942                        2067.9666   
                  LNEILQAR          9686                         955.5394   
E9PAV3            IEDLSQQAQLAAAEK   10645                       1613.8262   
                  NILFVITKPDVYK     17633                       1549.8953   
                                    17638                       1548.8916   

                                                 Observed M/Z  \
Protein Accession Peptide           Scan Number                 
B5ME19            CLEEFELLGK        16819            619.3103   
                  ELLGQGLLLR        17761            556.3480   
                  IMQNTDPHSQEYVEHLK 8942             518.0006   
                  LNEILQAR          9686             478.7803   
E9PAV3            IEDLSQQAQLAAAEK   10645            807.9237   
                  NILFVITKPDVYK     17633            517.6402   
                                    17638            775.4550   

                                                 Calibrated Observed M/Z  \
Protein Accession Peptide           Scan Number                            
B5ME19            CLEEFELLGK        16819                       619.3079   
                  ELLGQGLLLR        17761                       556.3457   
                  IMQNTDPHSQEYVEHLK 8942                        517.9989   
                  LNEILQAR          9686                        478.7770   
E9PAV3            IEDLSQQAQLAAAEK   10645                       807.920

#### multiIndex
This optional parameter allows you to create a custom multiIndex for your dataframe. The default multiIndicies are as follows:
*   1 mzML file: `['Scan Number']`
*   A dataframe which includes a PSM file: `['Protein Accession', 'Peptide', 'Scan Number']`
*   A peptide quantification file: `['Protein Accession', 'Peptide']`
*   1 protein quantification file: `['Protein Accession']`

 Custom MultiIndicies are inputted as a list (or str if it is a single index). MultiIndicies inputted as a list must be ordered according to the desired multiIndex hierarchy.

In [12]:
my_multiIndex = ['File Name', 'Protein Accession', 'Peptide', 'Scan Number']
custom_multiIndex_df = msfragger_parser(input_files=[msfragger_psm_file_path, msfragger_protein_file_path, msfragger_peptide_file_path], output_file_path=outfile, multiIndex=my_multiIndex)
custom_multiIndex_df.head()

Dataframe saved.


Modified Peptide  \
File Name                                       Protein Accession Peptide      Scan Number                    
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237                EQCEQLEK   
                                                A6NHQ2            TNIIPVLEDAR  14755                    NaN   
                                                A6NKT7            NSIPEPIDPLFK 16986                    NaN   
                                                A8MWD9            GNSIIMLEALER 18334                    NaN   
                                                B0I1T2            DFLFQDFK     18706                    NaN   

                                                                                            Charge  \
File Name                                       Protein Accession Peptide      Scan Number           
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237              2   
                                                A6NHQ2            TNIIPVLEDAR  14755             2   
                                                A6NKT7            NSIPEPIDPLFK 16986             2   
                                                A8MWD9            GNSIIMLEALER 18334             2   
                                                B0I1T2            DFLFQDFK     18706             2   

                                                                                            Retention  \
File Name                                       Protein Accession Peptide      Scan Number              
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237         1776.8865   
                                                A6NHQ2            TNIIPVLEDAR  14755        4218.5518   
                                                A6NKT7            NSIPEPIDPLFK 16986        4795.0662   
                                                A8MWD9            GNSIIMLEALER 18334        5152.8337   
                                                B0I1T2            DFLFQDFK     18706        5252.6033   

                                                                                            Observed Mass  \
File Name                                       Protein Accession Peptide      Scan Number                  
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237             1062.4706   
                                                A6NHQ2            TNIIPVLEDAR  14755            1239.6882   
                                                A6NKT7            NSIPEPIDPLFK 16986            1368.7386   
                                                A8MWD9            GNSIIMLEALER 18334            1344.7134   
                                                B0I1T2            DFLFQDFK     18706            1058.5103   

                                                                                            Calibrated Observed Mass  \
File Name                                       Protein Accession Peptide      Scan Number                             
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237                        1062.4664   
                                                A6NHQ2            TNIIPVLEDAR  14755                       1239.6842   
                                                A6NKT7            NSIPEPIDPLFK 16986                       1368.7316   
                                                A8MWD9            GNSIIMLEALER 18334                       1344.7035   
                                                B0I1T2            DFLFQDFK     18706                       1058.5061   

                                                                                            Observed M/Z  \
File Name                                       Protein Accession Peptide      Scan Number                 
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.p

It is also important to note that the multiIndex will not be saved in the output file. If you want to load a saved dataframe with a multiIndex, it can be done as follows:

In [13]:
loaded_df = pd.read_table(outfile).set_index(my_multiIndex).sort_index()
loaded_df.head()

Modified Peptide  \
File Name                                       Protein Accession Peptide      Scan Number                    
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237                EQCEQLEK   
                                                A6NHQ2            TNIIPVLEDAR  14755                    NaN   
                                                A6NKT7            NSIPEPIDPLFK 16986                    NaN   
                                                A8MWD9            GNSIIMLEALER 18334                    NaN   
                                                B0I1T2            DFLFQDFK     18706                    NaN   

                                                                                            Charge  \
File Name                                       Protein Accession Peptide      Scan Number           
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237              2   
                                                A6NHQ2            TNIIPVLEDAR  14755             2   
                                                A6NKT7            NSIPEPIDPLFK 16986             2   
                                                A8MWD9            GNSIIMLEALER 18334             2   
                                                B0I1T2            DFLFQDFK     18706             2   

                                                                                            Retention  \
File Name                                       Protein Accession Peptide      Scan Number              
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237         1776.8865   
                                                A6NHQ2            TNIIPVLEDAR  14755        4218.5518   
                                                A6NKT7            NSIPEPIDPLFK 16986        4795.0662   
                                                A8MWD9            GNSIIMLEALER 18334        5152.8337   
                                                B0I1T2            DFLFQDFK     18706        5252.6033   

                                                                                            Observed Mass  \
File Name                                       Protein Accession Peptide      Scan Number                  
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237             1062.4706   
                                                A6NHQ2            TNIIPVLEDAR  14755            1239.6882   
                                                A6NKT7            NSIPEPIDPLFK 16986            1368.7386   
                                                A8MWD9            GNSIIMLEALER 18334            1344.7134   
                                                B0I1T2            DFLFQDFK     18706            1058.5103   

                                                                                            Calibrated Observed Mass  \
File Name                                       Protein Accession Peptide      Scan Number                             
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.pep.xml A0A096LP55        EQCEQLEK     5237                        1062.4664   
                                                A6NHQ2            TNIIPVLEDAR  14755                       1239.6842   
                                                A6NKT7            NSIPEPIDPLFK 16986                       1368.7316   
                                                A8MWD9            GNSIIMLEALER 18334                       1344.7035   
                                                B0I1T2            DFLFQDFK     18706                       1058.5061   

                                                                                            Observed M/Z  \
File Name                                       Protein Accession Peptide      Scan Number                 
interact-Ex_Auto_J3_30umTB_02ngQC_60m_1.p

# Additional Info about Input Files
### mzML Files

**Default Columns**

Column Name | Definition | Source
------------| -----------| -------
base peak m/z | The m/z of the most intense peak in the spectrum | [source](http://www.chem.ucla.edu/harding/IGOC/B/base_peak.html#:~:text=Base%20peak%3A%20The%20most%20intense,are%20not%20always%20base%20peaks.) 
base peak intensity | The intensity of the most intense peak in the spectrum | [source](http://www.chem.ucla.edu/harding/IGOC/B/base_peak.html#:~:text=Base%20peak%3A%20The%20most%20intense,are%20not%20always%20base%20peaks.) 
total ion current | The sum of all the separate ion currents carried by the different ions contributing to the spectrum | [source]("http://mass-spec.lsu.edu/msterms/index.php/Total_ion_current#:~:text=Definitive%20Rules%2C%201979.-,Total%20ion%20current,ions%20contributing%20to%20the%20spectrum")
lowest observed m/z | Lowest m/z value observed in the m/z array. | [source](https://bioportal.bioontology.org/ontologies/MS?p=classes&conceptid=MS%3A1000528)
highest observed m/z | Highest m/z value observed in the m/z array. | [source](https://bioportal.bioontology.org/ontologies/MS?p=classes&conceptid=MS%3A1000528)
m/z array | An array containing observed m/z values from the spectrum. | Is this right? 
intensity array | An array containing observed intensity values from the spectrum. | Is this right?
selected ion m/z | ?
charge state | ?
peak intensity | ?

**Columns Deleted in MAPSI**

Column Name | Definition | Source
------------| -----------| -------
spectrum title |
count |
positive scan |
centroid spectrum |
defaultArrayLength |
MSn spectrum |
dataProcessingRef |
scanList |
MS1 spectrum |
ms level |


## MetaMorpheus
### PSM Files

Column Name | Definition 
------------| ----------|
File Name | mzML file name
Scan Retention Time | The experimental time (in minutes) that the scan was acquired. | [MM_wiki](https://github.com/smith-chem-wisc/MetaMorpheus/wiki/Glossary-of-Terminology)
Num Experimental Peaks | The number of experimental peaks (post-peak trimming) in the MS2 scan.
Total Ion Current | The total ion current of the MS2 spectrum. This is the sum of intensities from every MS2 peak. These intensities can come from fragmentation of multiple precursors depending on the selectivity for fragmentation (aka isolation width) and crowdedness of the MS1 spectrum.
Precursor Scan Number | The scan number of the most recent MS1 scan.
Precursor Charge | The charge of the isolated precursor peptide.
Precursor MZ | The mass to charge ratio of the isolated precursor peptide. This is not necessarily the selected MZ for isolation.
Precursor Mass |The neutral (uncharged) mass of the peptide.
Score | MetaMorpheus score is incremented by one for each matching b- and y-ion. c- and z.-ions are used for ETD. The number after the decimal is the fraction of total peak intensity from the MS2 scan that can be assigned to the particular peptide spectral match.
Delta Score | The MetaMorpheus score difference between the reported peptide and the next highest scoring peptide. If the next highest scoring peptide has the same score, both peptides are reported in the same row (ambiguity) and the next highest scoring peptide is used for the delta score. Thus, a delta score of 0 is not possible.
Notch | A narrow mass window in which the is an allowed mass difference between the experimentally observed peptide and the best matching theoretical peptide. This is an arbitrary number that signifies the notch's category.
Base Sequence | The peptide amino acid sequence without modifications.
Essential Sequence | The full sequence containing only database-defined modifications and absent of fixed/variable modifications.
Ambiguity Level | [see explanation](https://www.nature.com/articles/s41592-019-0573-x/tables/1)
PSM Count (unambiguous, <0.01 q-value) | 
Mods | The name(s) of the modification(s) on the peptide.
Mods Chemical Formulas | The chemical formula(s) of the identified modification(s).
Mods Combined Chemical Formula | The aggregated chemical formula of all identified modification.
Num Variable Mods | The number of variable modifications matched to a peptide.
Missed Cleavages | Digestion of proteins into peptides by proteases (e.g. trypsin) is occasionally incomplete. Users may specify the number of allowed missed cleavages to compensate for incomplete digestion of proteins. This expands the search space. Commonly, two missed cleavages are allowed. However, observation of missed cleavages should be expected as a rare event. For non-specific digestions, the number of missed cleavages is often reported as (the peptide length - 1).
Peptide Monoisotopic Mass | The mass of the peptide calculated from atoms in their most abundant isotopic form (12C, 16O, 14N, etc.). This is the uncharged (neutral) mass.
Mass Diff (Da) | The absolute mass difference between the observed and theoretical precursor mass. (Calculated as observed-theoretical).
Mass Diff (ppm) | The ppm mass difference between the observed and theoretical precursor mass. (Calculated as observed-theoretical).
Protein Name | The full name of the peptide's parent protein.
Gene Name | The gene name associated with the identified peptide's parent protein.
Organism Name | The database specified organism that the peptide's parent protein originated from.
Identified Sequence Variations | If the search was conducted using a database containing annotated sequence variants, this column displays the sequence variant that is identified by the PSM or peptide. To be considered as identified, the variant must satisfy one of two conditions. The first condition being that the peptide and variant must intersect creating a base sequence different than that which would be observed by digestion of the canonical, non-variant containing protein. The second condition is that the variant creates a proteolytic cleavage site which results in a peptide sequence that would not exist in the non-variant protein.
Splice Sites | If the search was conducted using a database that contains annotated splice sites, this column contains splice sites which the PSM or peptide crossed.
Contaminant | Specifies if the peptide's parent protein is from a contaminant database, "Y", or not "N".
Decoy | Specifies if the peptide is a decoy peptide "Y", or not "N".
Peptide Description | A brief statement regarding the peptide's digestion.
Start and End Residues In Protein | The one-based amino acid positions of the peptide in the parent protein(s).
Previous Amino Acid | The amino acid in the protein preceding the specified peptide.
Next Amino Acid | Amino acid in the protein that is next in line on the C-terminal end.
Theoreticals Searched | The number of theoretical peptides searched against the spectrum. This is only reported if e-Value calculations are specified in the search task.
Decoy/Contaminant/Target | Each peptide spectral match, unique peptide and protein is assigned as decoy (D)/contaminant (C)/or target (T). The preference in assignment is D>C>T.
Matched Ion Series | The found product ions and their respective charges.
Matched Ion Mass-To-Charge Ratios | The theoretical m/zs that were matched to the observed spectrum.
Matched Ion Mass Diff (Da) | The absolute mass differences between the observed and theoretical product ion masses. (Calculated as observed-theoretical). Order can be found in Matched Ion Series
Matched Ion Mass Diff (Ppm) | The ppm mass differences between the observed and theoretical product ion masses. (Calculated as observed-theoretical). Order can be found in Matched Ion Series
Matched Ion Intensities | The observed intensities for the matched product ions.
Matched Ion Counts | The number of product ions found for each series.
Normalized Spectral Angle | 
Localized Scores | If there is no ambiguity (only one peptide was assigned), then there is an attempt to localize the mass difference between the experimental and theoretical precursor masses. This mass difference is "placed" on each possible amino acid, and the resulting peptide score is calculated and reported in this column. Each reported score represents an amino acid (N-to-C) that the mass difference was "localized" to.
Improvement Possible | The increase in the MetaMorpheus score produced by localization of the modification to the position specified in the Full Sequence.
Cumulative Target | The target/decoy approach for determination of FDR yields lists of peptides and proteins matching either target or decoy. These commingled lists are sorted by score. The top scoring target match is labeled as 1. Each additional match to target is incremented by one. The total count of target matches scoring at or above a particular score at any point in the list is reported as Cumulative Target. Cumulative Decoy divided by Cumulative Target is the FDR.
Cumulative Decoy | The target/decoy approach for determination of FDR yields lists of peptides and proteins matching either target or decoy. These commingled lists are sorted by score. The top scoring decoy match is labeled as 1. Each additional match to decoy is incremented by one. The total count of decoy matches scoring at or above a particular score at any point in the list is reported as Cumulative Decoy. Cumulative Decoy divided by Cumulative Target is the FDR.
QValue | The q-value for the identification, calculated as the number of cumulative decoys (false positives) divided by the number of cumulative targets (true positives).
Cumulative Target Notch | The cumulative number of targets specific to the specified notch.
Cumulative Decoy Notch | The cumulative number of decoys specific to the specified notch.
QValue Notch |  The notch specific q-value for the identification, calculated as the number of cumulative notch decoys (false positives) divided by the number of cumulative notch targets (true positives).
PEP | 
PEP_QValue | 

[source](https://github.com/smith-chem-wisc/MetaMorpheus/wiki/Glossary-of-Terminology)


### Peptide Quantification


Column Name | Definition |
------------| -----------|
Base Sequence | The peptide amino acid sequence without modifications.
Gene Names | The gene name associated with the identified peptide's parent protein.
Organism
Intensity_Ex_Auto_J3_30umTB_2ngQC_60m_1-calib
Intensity_Ex_Auto_J3_30umTB_2ngQC_60m_2-calib
Intensity_Ex_Auto_K13_30umTA_2ngQC_60m_1-calib
Intensity_Ex_Auto_K13_30umTA_2ngQC_60m_2-calib
Intensity_Ex_Auto_W17_30umTB_2ngQC_60m_1-calib
Intensity_Ex_Auto_W17_30umTB_2ngQC_60m_2-calib
Detection Type_Ex_Auto_J3_30umTB_2ngQC_60m_1-calib
Detection Type_Ex_Auto_J3_30umTB_2ngQC_60m_2-calib
Detection Type_Ex_Auto_K13_30umTA_2ngQC_60m_1-calib
Detection Type_Ex_Auto_K13_30umTA_2ngQC_60m_2-calib
Detection Type_Ex_Auto_W17_30umTB_2ngQC_60m_1-calib
Detection Type_Ex_Auto_W17_30umTB_2ngQC_60m_2-calib

[source](https://github.com/smith-chem-wisc/MetaMorpheus/wiki/Glossary-of-Terminology)

### Protein Quantification

Column Name | Definition |
------------| -----------|
Gene | The gene name associated with the protein.
Organism
Protein Full Name | The full name of the protein as specified in the protein database.
Protein Unmodified Mass
Number of Proteins in Group | The number of proteins in the protein group. Multiple proteins are associated with a peptide identification when parsimony cannot distinguish between the options.
Unique Peptides | Peptides that are unique to the listed protein (they can only come from that one protein, based on the database in silico digestion). Currently, peptides that are unique to the group are not listed here; i.e., a protein group with >1 protein will always have 0 unique peptides because they are shared between all proteins in the group.
Shared Peptides | Peptides that are shared between multiple proteins in the protein database(s) used for the search are listed.
Number of Peptides | Number of unique+shared peptides observed that match to the specified protein group.
Number of Unique Peptides | Number of unique proteins for the protein group. See Unique Peptides definition.
Sequence Coverage Fraction | The fraction of amino acids in the protein observed in any peptide spectral match with a Q value <0.01.
Sequence Coverage | Displays amino acids in the protein observed in any peptide spectral match with a Q value <0.01 for each protein in the group, with the "|" character as the delimiter. Lowercase residues were not observed. Uppercase residues were observed.
Sequence Coverage with Mods
Modification Info List
Intensity_Ex_Auto_J3_30umTB_2ngQC_60m_1-calib
Intensity_Ex_Auto_J3_30umTB_2ngQC_60m_2-calib
Intensity_Ex_Auto_K13_30umTA_2ngQC_60m_1-calib
Intensity_Ex_Auto_K13_30umTA_2ngQC_60m_2-calib
Intensity_Ex_Auto_W17_30umTB_2ngQC_60m_1-calib
Intensity_Ex_Auto_W17_30umTB_2ngQC_60m_2-calib
Number of PSMs | The number of peptide spectral matches below with a Q-Value <0.01 observed for all peptides assigned to the protein group.
Protein Decoy/Contaminant/Target
Protein Cumulative Target
Protein Cumulative Decoy
Protein QValue
Best Peptide Score
Best Peptide Notch QValue


[source](https://github.com/smith-chem-wisc/MetaMorpheus/wiki/Glossary-of-Terminology)

## MSFragger
### PSM Files

Column Name | Definition |
------------| -----------|
File Name | mzML file name?
Modified Peptide | 
Prev AA | 
Next AA
Peptide Length
Charge
Retention
Observed Mass
Calibrated Observed Mass
Observed M/Z
Calibrated Observed M/Z
Calculated Peptide Mass
Calculated M/Z
Delta Mass
Expectation
Hyperscore
Nextscore
PeptideProphet Probability
Number of Enzymatic Termini
Number of Missed Cleavages
Protein Start
Protein End
Intensity
Assigned Modifications
Observed Modifications
Is Unique
Protein
Entry Name
Gene
Protein Description

### Peptide Quantification

Column Name | Definition |
------------| -----------|
Prev AA
Next AA
Start
End
Peptide Length
Charges
Protein
Entry Name
Gene
Protein Description
Mapped Genes
Mapped Proteins
Ex_Auto_J3_30umTB_02ngQC_60m_1 Spectral Count
Ex_Auto_J3_30umTB_02ngQC_60m_2 Spectral Count
Ex_Auto_K13_30umTA_02ngQC_60m_1 Spectral Count
Ex_Auto_K13_30umTA_02ngQC_60m_2 Spectral Count
Ex_Auto_W17_30umTA_02ngQC_60m_3 Spectral Count
Ex_Auto_W17_30umTA_02ngQC_60m_4 Spectral Count
Ex_Auto_J3_30umTB_02ngQC_60m_1 Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_2 Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_1 Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_2 Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_3 Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_4 Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_1 MaxLFQ Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_2 MaxLFQ Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_1 MaxLFQ Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_2 MaxLFQ Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_3 MaxLFQ Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_4 MaxLFQ Intensity

### Protein Quantification

Column Name | Definition |
------------| -----------|
Protein
Entry Name
Gene
Protein Length
Coverage
Organism
Protein Existence
Description
Protein Probability
Top Peptide Probability
Combined Total Peptides
Combined Spectral Count
Combined Unique Spectral Count
Combined Total Spectral Count
Ex_Auto_J3_30umTB_02ngQC_60m_1 Spectral Count
Ex_Auto_J3_30umTB_02ngQC_60m_2 Spectral Count
Ex_Auto_K13_30umTA_02ngQC_60m_1 Spectral Count
Ex_Auto_K13_30umTA_02ngQC_60m_2 Spectral Count
Ex_Auto_W17_30umTA_02ngQC_60m_3 Spectral Count
Ex_Auto_W17_30umTA_02ngQC_60m_4 Spectral Count
Ex_Auto_J3_30umTB_02ngQC_60m_1 Unique Spectral Count
Ex_Auto_J3_30umTB_02ngQC_60m_2 Unique Spectral Count
Ex_Auto_K13_30umTA_02ngQC_60m_1 Unique Spectral Count
Ex_Auto_K13_30umTA_02ngQC_60m_2 Unique Spectral Count
Ex_Auto_W17_30umTA_02ngQC_60m_3 Unique Spectral Count
Ex_Auto_W17_30umTA_02ngQC_60m_4 Unique Spectral Count
Ex_Auto_J3_30umTB_02ngQC_60m_1 Total Spectral Count
Ex_Auto_J3_30umTB_02ngQC_60m_2 Total Spectral Count
Ex_Auto_K13_30umTA_02ngQC_60m_1 Total Spectral Count
Ex_Auto_K13_30umTA_02ngQC_60m_2 Total Spectral Count
Ex_Auto_W17_30umTA_02ngQC_60m_3 Total Spectral Count
Ex_Auto_W17_30umTA_02ngQC_60m_4 Total Spectral Count
Ex_Auto_J3_30umTB_02ngQC_60m_1 Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_2 Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_1 Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_2 Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_3 Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_4 Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_1 Unique Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_2 Unique Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_1 Unique Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_2 Unique Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_3 Unique Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_4 Unique Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_1 Total Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_2 Total Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_1 Total Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_2 Total Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_3 Total Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_4 Total Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_1 MaxLFQ Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_2 MaxLFQ Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_1 MaxLFQ Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_2 MaxLFQ Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_3 MaxLFQ Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_4 MaxLFQ Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_1 MaxLFQ Unique Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_2 MaxLFQ Unique Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_1 MaxLFQ Unique Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_2 MaxLFQ Unique Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_3 MaxLFQ Unique Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_4 MaxLFQ Unique Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_1 MaxLFQ Total Intensity
Ex_Auto_J3_30umTB_02ngQC_60m_2 MaxLFQ Total Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_1 MaxLFQ Total Intensity
Ex_Auto_K13_30umTA_02ngQC_60m_2 MaxLFQ Total Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_3 MaxLFQ Total Intensity
Ex_Auto_W17_30umTA_02ngQC_60m_4 MaxLFQ Total Intensity
Indistinguishable Proteins